In [ ]:
import numpy as np
import math as m
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
#from scipy.linalg import sqrtm
from tqdm import tqdm
import torch

In [ ]:
def roundc(c, digits):
    if c.imag == 0:
        return round(c.real, digits)
    else:
        return round(c.real, digits) + round(c.imag, digits) * 1j

In [ ]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states(coeff, Abstract = False):    # coeff list like [a0,a1,b0,b1]
    if Abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [ ]:
# creating the SIC POVM matrices
w = m.e**((2/3)*m.pi*(1j))
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrix

M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition

u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix


In [ ]:
def fnj_vec(N, coeff, priors, POVM_elts):

    initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated

    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    psi0psi0 = [psi0sq[0], np.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]    
    psi1psi1 = [psi1sq[0], np.sqrt(psi1sq[1]**(2)+psi1sq[2]**(2)), psi1sq[3]]    # creating square states
    
    vec_psi0psi0 = np.array(psi0psi0)     
    vec_psi1psi1 = np.array(psi1psi1)
    rho = priors[0]*np.outer(vec_psi0psi0, vec_psi0psi0)+ priors[1]*np.outer(vec_psi1psi1, vec_psi1psi1)     # theoretical density matrix with priors 1/2 each.

    prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]

    return nj_vec

In [ ]:
#Creating function for the deltarho error

def experiment_rho(N, coeff, priors, POVM_elts, M_inv):

    initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated

    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    psi0psi0 = [psi0sq[0], np.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]    
    psi1psi1 = [psi1sq[0], np.sqrt(psi1sq[1]**(2)+psi1sq[2]**(2)), psi1sq[3]]    # creating square states
    
    vec_psi0psi0 = np.array(psi0psi0)     
    vec_psi1psi1 = np.array(psi1psi1)
    rho = priors[0]*np.outer(vec_psi0psi0, vec_psi0psi0)+ priors[1]*np.outer(vec_psi1psi1, vec_psi1psi1)     # theoretical density matrix with priors 1/2 each.

    prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     
    
    r_vec = np.dot(M_inv,pj_num_vec)
    
    # constructing the rho numrical using the r_vector etc.
    rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]   # list of matrices, see equation 7 in Hillery notes pair_disc.pdf
    rho_num = np.zeros_like(rho_num_list[0])
    for matrix in rho_num_list:
        rho_num = np.add(rho_num, matrix)       # created the numerical rho
    
    del_rho = rho_num-rho 
    
    eigenvalues, eigenvectors = np.linalg.eig(del_rho)  # each COLUMN is the corresponding eigenvector and not the row. eigenvectors[:,0]
    # eigenvectors[:,0]   # eigenvector correspoding to the first eigenvalue
    evals = np.array([i.real for i in eigenvalues if abs(i.imag)< .001])
    norm_del_rho = max([abs(i) for i in evals])     # picking out the largest magnitude evalue
    error = norm_del_rho

    return error

In [ ]:

coeff = [1j/(2**.5),1j/(2**.5),1j/(2**.5),1j/(2**.5)] #1/(2**.5)
priors = [.5,.5]
trials = [100*(i+1) for i in range(100)]

sqrtN  = [2.6/(i**.5) for i in trials]     # ideal square root shape with a constant multiplied for scaling
errors = [experiment_rho(i,coeff,priors,POVM_elts, M_inv) for i in tqdm(trials)]

plt.plot(trials, errors, label=r'$\Delta \rho $')
plt.plot(trials ,sqrtN, "--", label=r'$\frac{1}{\sqrt{N}}$')
plt.xlabel(' N (trials)')
plt.ylabel(r'$\Delta \rho $')
plt.title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_0={},\,p_1={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), roundc(coeff[2],1), roundc(coeff[3],1), priors[0], priors[1]))
plt.legend()

plt.show()

print(f'Error Converged to: ' , experiment_rho( 1000*100 ,coeff,priors,POVM_elts, M_inv))

In [ ]:

#Creating function for the fidelity error

def solve_quadratic(a, b, c):

    if a == 0 and c == 0:
        sol1 = 0
        sol2 = 0

    dis = b * b - 4 * a * c 
    sqrt_val = m.sqrt(abs(dis))

    if dis > 0: 
        sol1 = (-b - sqrt_val)/(2 * a)
        sol2 = (-b + sqrt_val)/(2 * a)
     
    elif dis == 0: 
        sol1 = -b / (2 * a)
        sol2 = -b / (2 * a)
     
    # when discriminant is less than 0
    else:
        sol1 = - b / (2 * a) - 1j*sqrt_val/ (2 * a)
        sol2 = - b / (2 * a) + 1j*sqrt_val/ (2 * a)
        
    return sol1, sol2
    
    '''coefficients = np.array([a, b, c])
    roots = np.roots(coefficients)
    return roots'''

def experiment_fid(N, coeff, priors, POVM_elts, M_inv):
    initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated

    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    psi0psi0 = [psi0sq[0], np.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]    
    psi1psi1 = [psi1sq[0], np.sqrt(psi1sq[1]**(2)+psi1sq[2]**(2)), psi1sq[3]]    # creating square states
    
    vec_psi0psi0 = np.array(psi0psi0)     
    vec_psi1psi1 = np.array(psi1psi1)
    rho = priors[0]*np.outer(vec_psi0psi0, vec_psi0psi0)+ priors[1]*np.outer(vec_psi1psi1, vec_psi1psi1)     # theoretical density matrix with priors 1/2 each.

    prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     
    
    r_vec = np.dot(M_inv,pj_num_vec)
    
    # constructing the rho numrical using the r_vector etc.
    rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]   # list of matrices, see equation 7 in Hillery notes pair_disc.pdf
    rho_num = np.zeros_like(rho_num_list[0])
    for matrix in rho_num_list:
        rho_num = np.add(rho_num, matrix)       # created the numerical rho

    #print(f'rho_num: {rho_num}')
     
    eigenvalues, eigenvectors = np.linalg.eig(rho_num)
    evals = np.array([i.real for i in eigenvalues if abs(i.imag)< .001])
    #evals = np.array([abs(i) for i in eigenvalues])
    index = np.argmin(evals)
    eigenvector = eigenvectors[:,index]

    c00=eigenvector[0]
    c01=eigenvector[1]
    c01t=eigenvector[1]/m.sqrt(2)
    c11=eigenvector[2]

    #print(f'c00: {c00}, c01: {c01}, c01t: {c01t}, c11: {c11}')

    f0,f1=solve_quadratic(c00, 2*c01t, c11)
    f0p,f1p=solve_quadratic(c11, 2*c01t, c00)

    #print(f'f0: {f0}, f1: {f1}, f0p: {f0p}, f1p: {f1p}')
    #print(f'abs(f0): {abs(f0)**2}, abs(f1): {abs(f1)**2}, abs(f0p): {abs(f0p)**2}, abs(f1p): {abs(f1p)**2}')
    #print(f'sqrt(1+abs(f0)): {m.sqrt(1+abs(f0)**2)}, sqrt(1+abs(f1)): {m.sqrt(1+abs(f1)**2)}, sqrt(1+abs(f0p)): {m.sqrt(1+abs(f0p)**2)}, sqrt(1+abs(f1p)): {m.sqrt(1+abs(f1p)**2)}')
    #print(f'1/(sqrt(1+abs(f0))): {1/(m.sqrt(1+abs(f0)**2))}, 1/(sqrt(1+abs(f1))): {1/(m.sqrt(1+abs(f1)**2))}, 1/(sqrt(1+abs(f0p))): {1/(m.sqrt(1+abs(f0p)**2))}, 1/(sqrt(1+abs(f1p))): {1/(m.sqrt(1+abs(f1p)**2))}')
    #print(f'abs(f0)/(sqrt(1+abs(f0))): {abs(f0)/(m.sqrt(1+abs(f0)**2))}, abs(f1)/(sqrt(1+abs(f1))): {abs(f1)/(m.sqrt(1+abs(f1)**2))}, abs(f0p)/(sqrt(1+abs(f0p))): {abs(f0p)/(m.sqrt(1+abs(f0p)**2))}, abs(f1p)/(sqrt(1+abs(f1p))): {abs(f1p)/(m.sqrt(1+abs(f1p)**2))}')

    #csi orthogonal vector and its components
    csi = np.asarray([c00,c01,c11])
    csi0 = abs(np.vdot(csi,psi0psi0))**2
    csi1 = abs(np.vdot(csi,psi1psi1))**2
    csis = [csi0,csi1]

        
    #all possible (positive) coeffs
    a1 = np.conj(f0)/(m.sqrt(1+ (abs(f0))**2)) 
    a2 = 1/(m.sqrt(1+ (abs(f0))**2))
    a3 = np.conj(f0p)/(m.sqrt(1+ (abs(f0p))**2)) 
    a4 = 1/(m.sqrt(1+ (abs(f0p))**2))
    a5 = np.conj(f1)/(m.sqrt(1+ (abs(f1))**2)) 
    a6 = 1/(m.sqrt(1+ (abs(f1))**2))
    a7 = np.conj(f1p)/(m.sqrt(1+ (abs(f1p))**2))
    a8 = 1/(m.sqrt(1+ (abs(f1p))**2))

    #lists of all possible coeffs, including megative and complex conjugates. each sublist correspons to a "physical sector" 
    v12 = [a1,a2,-a1,-a2]
    v34 = [a3,a4,-a3,-a4]
    v56 = [a5,a6,-a5,-a6]
    v78 = [a7,a8,-a7,-a8]

    pairs12 = [[i,j] for i in v12 for j in v12 if i!=j and i!= -j]
    #print(f'len(pairs12): {len(pairs12)}') 
    pairs34 = [[i,j] for i in v34 for j in v34 if i!=j and i!= -j]
    #print(f'len(pairs34): {len(pairs34)}')
    pairs56 = [[i,j] for i in v56 for j in v56 if i!=j and i!= -j] 
    #print(f'len(pairs56): {len(pairs56)}')
    pairs78 = [[i,j] for i in v78 for j in v78 if i!=j and i!= -j]
    #print(f'len(pairs78): {len(pairs78)}')

    pairs = pairs12 + pairs34 + pairs56 + pairs78

    #print(f'pairs: {pairs}')
    
    #all possible fidelities
    fid_psi0 = [abs(np.vdot(i, psi0))**2 for i in pairs]
    fid_psi1 = [abs(np.vdot(j, psi1))**2 for j in pairs]

    #Create the dictionaries for the fidelities and the corresponding num wavefunctions
    fid_psi0_dict = dict(zip(fid_psi0, pairs))
    fid_psi1_dict = dict(zip(fid_psi1, pairs))
    
    #find the max fidelity and the corresponding num wavefunction
    fid0 = max(fid_psi0_dict.keys())
    psi_num0 = fid_psi0_dict[fid0]
    fid1 = max(fid_psi1_dict.keys())
    psi_num1 = fid_psi1_dict[fid1]

    #print(f'fid0: {fid0}, fid1: {fid1}')
    #print(f'psi_0: {psi_num0}, psi_1: {psi_num1}')


    #extract p0 and p1 numerical from rho_num
    psi0sqn = []
    psi1sqn = []
    [[psi0sqn.append(i*j) for i in psi_num0] for j in psi_num0]
    [[psi1sqn.append(i*j) for i in psi_num1] for j in psi_num1]
    #print(f'psi0sqn: {psi0sqn}, psi1sqn: {psi1sqn}')   
    psi0psi0n = [psi0sqn[0], np.sqrt(psi0sqn[1]**2+psi0sqn[2]**2), psi0sqn[3]]    
    psi1psi1n = [psi1sqn[0], np.sqrt(psi1sqn[1]**(2)+psi1sqn[2]**(2)), psi1sqn[3]]    # creating square states
    #print(f'psi0psi0n: {psi0psi0n}, psi1psi1n: {psi1psi1n}')
    vec_psi0psi0n = np.array(psi0psi0n)     
    vec_psi1psi1n = np.array(psi1psi1n)
    #print(f'vec_psi0psi0n: {vec_psi0psi0n}, vec_psi1psi1n: {vec_psi1psi1n}')

    #print(f'rho_num: {rho_num}')
    p0c = (np.vdot(vec_psi0psi0n,rho_num @ vec_psi0psi0n) - abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)/(1-abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)
    p1c = (np.vdot(vec_psi1psi1n,rho_num @ vec_psi1psi1n) - abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)/(1-abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)
    #print(f'rho|psi0psi0>: {np.dot(rho_num, vec_psi0psi0n)}')
    #print(f'rho|psi1psi1>: {np.dot(rho_num, vec_psi1psi1n)}')


    #print(f'<psi0psi0|rho|psi0psi0>: {np.vdot(vec_psi0psi0n,rho_num @ vec_psi0psi0n)}')
    #print(f'<psi1psi1|rho|psi1psi1>: {np.vdot(vec_psi1psi1n,rho_num @ vec_psi1psi1n)}')
    #print(f'|<psi0psi0|psi1psi1>|^2: {abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2}')

    #print(f'p0c: {p0c}, p1c: {p1c}')

    p0 = abs(p0c)
    p1 = abs(p1c)

    fid = [fid0,fid1]
    psin = [psi_num0,psi_num1]
    p = [p0,p1]

    #print(f'fid: {fid}, psin: {psin}, p: {p}')
    ##print(f'fid: {fid}, psin: {psin}, p: {p}')
    #print(f'norm of psin: {np.linalg.norm(psin[0])}, {np.linalg.norm(psin[1])}')

    return([fid,csis,csi,psin,p])

In [ ]:
'''     
    # Find the tuple that minimizes (x - psi0[0])**2 + (y - psi0[1])**2
    min_value0 = min(pairs, key=lambda x: (x[0] - psi0[0])**2 + (x[1] - psi0[1])**2)
    # Find the tuple that minimizes (x - psi1[0])**2 + (y - psi1[1])**2
    min_value1 = min(pairs, key=lambda x: (x[0] - psi1[0])**2 + (x[1] - psi1[1])**2)
    psi_num0_nn = list(min_value0)
    psi_num1_nn = list(min_value1)

    psi_num0 = psi_num0_nn/np.linalg.norm(psi_num0_nn)
    psi_num1 = psi_num1_nn/np.linalg.norm(psi_num1_nn)

    fid0 = abs(np.vdot(psi_num0,psi0))**2
    fid1 = abs(np.vdot(psi_num1,psi1))**2
'''

        
''' # Find the value closest to psi0,1[0,1], norm L2
    
    values = [a0,a1,b0,b1,a0p,a1p,b0p,b1p]
    cs_a0 = min(values, key=lambda x: (x - psi0[0])**2)
    cs_a1 = min(values, key=lambda x: (x - psi0[1])**2)
    cs_b0 = min(values, key=lambda x: (x - psi1[0])**2)
    cs_b1 = min(values, key=lambda x: (x - psi1[1])**2)
    psi_num0 = [cs_a0,cs_a1]
    psi_num1 = [cs_b0,cs_b1]
    
    fid0 = abs(np.vdot(psi_num0, psi0))**2
    fid1 = abs(np.vdot(psi_num1, psi1))**2



'''

In [ ]:
def grad_fid(N, coeff, priors, POVM_elts, M_inv):
    initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated

    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    psi0psi0 = [psi0sq[0], np.sqrt(psi0sq[1]**2+psi0sq[2]**2), psi0sq[3]]    
    psi1psi1 = [psi1sq[0], np.sqrt(psi1sq[1]**(2)+psi1sq[2]**(2)), psi1sq[3]]    # creating square states
    
    vec_psi0psi0 = np.array(psi0psi0)     
    vec_psi1psi1 = np.array(psi1psi1)
    rho = priors[0]*np.outer(vec_psi0psi0, np.conj(vec_psi0psi0)) + priors[1]*np.outer(vec_psi1psi1, np.conj(vec_psi1psi1))     # theoretical density matrix with priors 1/2 each.

    prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     
    
    r_vec = np.dot(M_inv,pj_num_vec)
    
    # constructing the rho numrical using the r_vector etc.
    rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]   # list of matrices, see equation 7 in Hillery notes pair_disc.pdf
    rho_num = np.zeros_like(rho_num_list[0])
    for matrix in rho_num_list:
        rho_num = np.add(rho_num, matrix)       # created the numerical rho

    #print(f'rho_num: {rho_num}')
     
    eigenvalues, eigenvectors = np.linalg.eig(rho_num)
    evals = np.array([i.real for i in eigenvalues if abs(i.imag)< .001])
    #evals = np.array([abs(i) for i in eigenvalues])
    index = np.argmin(evals)
    eigenvector = eigenvectors[:,index]

    c00=eigenvector[0]
    c01=eigenvector[1]
    c01t=eigenvector[1]/m.sqrt(2)
    c11=eigenvector[2]

    #print(f'c00: {c00}, c01: {c01}, c01t: {c01t}, c11: {c11}')

    f0,f1=solve_quadratic(c00, 2*c01t, c11)
    f0p,f1p=solve_quadratic(c11, 2*c01t, c00)

    #print(f'f0: {f0}, f1: {f1}, f0p: {f0p}, f1p: {f1p}')
    #print(f'abs(f0): {abs(f0)**2}, abs(f1): {abs(f1)**2}, abs(f0p): {abs(f0p)**2}, abs(f1p): {abs(f1p)**2}')
    #print(f'sqrt(1+abs(f0)): {m.sqrt(1+abs(f0)**2)}, sqrt(1+abs(f1)): {m.sqrt(1+abs(f1)**2)}, sqrt(1+abs(f0p)): {m.sqrt(1+abs(f0p)**2)}, sqrt(1+abs(f1p)): {m.sqrt(1+abs(f1p)**2)}')
    #print(f'1/(sqrt(1+abs(f0))): {1/(m.sqrt(1+abs(f0)**2))}, 1/(sqrt(1+abs(f1))): {1/(m.sqrt(1+abs(f1)**2))}, 1/(sqrt(1+abs(f0p))): {1/(m.sqrt(1+abs(f0p)**2))}, 1/(sqrt(1+abs(f1p))): {1/(m.sqrt(1+abs(f1p)**2))}')
    #print(f'abs(f0)/(sqrt(1+abs(f0))): {abs(f0)/(m.sqrt(1+abs(f0)**2))}, abs(f1)/(sqrt(1+abs(f1))): {abs(f1)/(m.sqrt(1+abs(f1)**2))}, abs(f0p)/(sqrt(1+abs(f0p))): {abs(f0p)/(m.sqrt(1+abs(f0p)**2))}, abs(f1p)/(sqrt(1+abs(f1p))): {abs(f1p)/(m.sqrt(1+abs(f1p)**2))}')

    #csi orthogonal vector and its components
    csi = np.asarray([c00,c01,c11])
    csi0 = abs(np.vdot(csi,psi0psi0))**2
    csi1 = abs(np.vdot(csi,psi1psi1))**2
    csis = [csi0,csi1]

        
    #all possible (positive) coeffs
    a1 = np.conj(f0)/(m.sqrt(1+ (abs(f0))**2)) 
    a2 = 1/(m.sqrt(1+ (abs(f0))**2))
    a3 = np.conj(f0p)/(m.sqrt(1+ (abs(f0p))**2)) 
    a4 = 1/(m.sqrt(1+ (abs(f0p))**2))
    a5 = np.conj(f1)/(m.sqrt(1+ (abs(f1))**2)) 
    a6 = 1/(m.sqrt(1+ (abs(f1))**2))
    a7 = np.conj(f1p)/(m.sqrt(1+ (abs(f1p))**2))
    a8 = 1/(m.sqrt(1+ (abs(f1p))**2))

    #lists of all possible coeffs, including megative and complex conjugates. each sublist correspons to a "physical sector" 
    v12 = [a1,a2,-a1,-a2]
    v34 = [a3,a4,-a3,-a4]
    v56 = [a5,a6,-a5,-a6]
    v78 = [a7,a8,-a7,-a8]

    pairs12 = [[i,j] for i in v12 for j in v12 if i!=j and i!= -j]
    #print(f'len(pairs12): {len(pairs12)}') 
    pairs34 = [[i,j] for i in v34 for j in v34 if i!=j and i!= -j]
    #print(f'len(pairs34): {len(pairs34)}')
    pairs56 = [[i,j] for i in v56 for j in v56 if i!=j and i!= -j] 
    #print(f'len(pairs56): {len(pairs56)}')
    pairs78 = [[i,j] for i in v78 for j in v78 if i!=j and i!= -j]
    #print(f'len(pairs78): {len(pairs78)}')

    pairs = pairs12 + pairs34 + pairs56 + pairs78

    #print(f'pairs: {pairs}')
    
    #all possible fidelities
    fid_psi0 = [abs(np.vdot(i, psi0))**2 for i in pairs]
    fid_psi1 = [abs(np.vdot(j, psi1))**2 for j in pairs]

    #Create the dictionaries for the fidelities and the corresponding num wavefunctions
    fid_psi0_dict = dict(zip(fid_psi0, pairs))
    fid_psi1_dict = dict(zip(fid_psi1, pairs))
    
    #find the max fidelity and the corresponding num wavefunction
    fid0 = max(fid_psi0_dict.keys())
    psi_num0 = fid_psi0_dict[fid0]
    fid1 = max(fid_psi1_dict.keys())
    psi_num1 = fid_psi1_dict[fid1]

    #print(f'fid0: {fid0}, fid1: {fid1}')
    #print(f'psi_0: {psi_num0}, psi_1: {psi_num1}')


    #extract p0 and p1 numerical from rho_num
    psi0sqn = []
    psi1sqn = []
    [[psi0sqn.append(i*j) for i in psi_num0] for j in psi_num0]
    [[psi1sqn.append(i*j) for i in psi_num1] for j in psi_num1]
    #print(f'psi0sqn: {psi0sqn}, psi1sqn: {psi1sqn}')   
    psi0psi0n = [psi0sqn[0], np.sqrt(psi0sqn[1]**2+psi0sqn[2]**2), psi0sqn[3]]    
    psi1psi1n = [psi1sqn[0], np.sqrt(psi1sqn[1]**(2)+psi1sqn[2]**(2)), psi1sqn[3]]    # creating square states
    #print(f'psi0psi0n: {psi0psi0n}, psi1psi1n: {psi1psi1n}')
    vec_psi0psi0n = np.array(psi0psi0n)     
    vec_psi1psi1n = np.array(psi1psi1n)
    #print(f'vec_psi0psi0n: {vec_psi0psi0n}, vec_psi1psi1n: {vec_psi1psi1n}')

    #print(f'rho_num: {rho_num}')
    p0c = (np.vdot(vec_psi0psi0n,rho_num @ vec_psi0psi0n) - abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)/(1-abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)
    p1c = (np.vdot(vec_psi1psi1n,rho_num @ vec_psi1psi1n) - abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)/(1-abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2)
    #print(f'rho|psi0psi0>: {np.dot(rho_num, vec_psi0psi0n)}')
    #print(f'rho|psi1psi1>: {np.dot(rho_num, vec_psi1psi1n)}')


    #print(f'<psi0psi0|rho|psi0psi0>: {np.vdot(vec_psi0psi0n,rho_num @ vec_psi0psi0n)}')
    #print(f'<psi1psi1|rho|psi1psi1>: {np.vdot(vec_psi1psi1n,rho_num @ vec_psi1psi1n)}')
    #print(f'|<psi0psi0|psi1psi1>|^2: {abs(np.vdot(vec_psi0psi0n,vec_psi1psi1n))**2}')

    #print(f'p0c: {p0c}, p1c: {p1c}')

    p0 = abs(p0c)
    p1 = abs(p1c)

    fid = [fid0,fid1]
    psin = [psi_num0,psi_num1]
    p = [p0,p1]

    #print(f'fid: {fid}, psin: {psin}, p: {p}')
    ##print(f'fid: {fid}, psin: {psin}, p: {p}')
    #print(f'norm of psin: {np.linalg.norm(psin[0])}, {np.linalg.norm(psin[1])}')

    return([fid,csis,csi,psin,p])

In [ ]:
def englert_fid(N, coeff, priors, POVM_elts, M_inv):
    initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
    psi0 = initial_states[0]
    psi1 = initial_states[1]    # created the states to be discriminated

    #print(f'psi0: {psi0}, psi1: {psi1}')

    psi0sq = []
    psi1sq = []
    [[psi0sq.append(i*j) for i in psi0] for j in psi0]
    [[psi1sq.append(i*j) for i in psi1] for j in psi1]   
    psi0psi0 = [psi0sq[0], (np.sqrt(psi0sq[1]**2+psi0sq[2]**2)), psi0sq[3]]    
    psi1psi1 = [psi1sq[0],(np.sqrt(psi1sq[1]**(2)+psi1sq[2]**(2))), psi1sq[3]]    # creating square states
    
    vec_psi0psi0 = np.array(psi0psi0)     
    vec_psi1psi1 = np.array(psi1psi1)
    rho = priors[0]*np.outer(vec_psi0psi0, np.conj(vec_psi0psi0))+ priors[1]*np.outer(vec_psi1psi1, np.conj(vec_psi1psi1))     # theoretical density matrix with priors 1/2 each.

    prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] 
    prob_vec = [i.real for i in prob_vec if abs(i.imag) < .01]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     
    
    r_vec = np.dot(M_inv,pj_num_vec)
    
    # constructing the rho numrical using the r_vector etc.
    rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]   # list of matrices, see equation 7 in Hillery notes pair_disc.pdf
    rho_num = np.zeros_like(rho_num_list[0])
    for matrix in rho_num_list:
        rho_num = np.add(rho_num, matrix)       # created the numerical rho

    #print(f'rho_num: {rho_num}')  
    
    #relations between s and C and the components of rho_num - proven in the notes
    s = np.empty(3,dtype=complex)
    C = np.empty((3,3),dtype=complex)
    
    s[0] = (1/(np.sqrt(2)))*(rho_num[0,2] + rho_num[1,2] + rho_num[2,0] + rho_num[2,1])
    s[1] = (1/(np.sqrt(2)))*(rho_num[0,2] - rho_num[1,2] - rho_num[2,0] + rho_num[2,1])
    s[2] = rho_num[0,0] - rho_num[1,1]

    C[0,0] = rho_num[0,1] + rho_num[1,0] + rho_num[2,2]
    C[0,1] = 1j*(rho_num[0,1] - rho_num[1,0])
    C[0,2] = (1/(np.sqrt(2)))*(rho_num[0,2] - rho_num[1,2] + rho_num[2,0] - rho_num[2,1])
    C[1,0] = 1j*(rho_num[0,1] - rho_num[1,0]) #=C[0,1]
    C[1,1] = -rho_num[0,1] - rho_num[1,0] + rho_num[2,2]
    C[1,2] = (1j/(np.sqrt(2)))*(rho_num[0,2] + rho_num[1,2] - rho_num[2,0] - rho_num[2,1])
    C[2,0] = (1/(np.sqrt(2)))*(rho_num[0,2] - rho_num[1,2] + rho_num[2,0] - rho_num[2,1]) #=C[0,2]
    C[2,1] = (1j/(np.sqrt(2)))*(rho_num[0,2] + rho_num[1,2] - rho_num[2,0] - rho_num[2,1]) #=C[1,2]
    C[2,2] = rho_num[0,0] + rho_num[1,1] - rho_num[2,2]

    #print(f's: {s}, C: {C}')
    #print(f'Cs: {C@s}')
    #print(f's-sC: {s-C@s}')
    #print(f's^2: {np.vdot(s,s)}')
    #print(f'1-s^2: {1-np.vdot(s,s)}')
    

    #r0 = np.empty(3,dtype=complex)
    #r1 = np.empty(3,dtype=complex)

    '''if np.vdot(s,s) == 1:
        #print('s is a=b')
        #how to find p0,p1 in this case? let's generate some random ones in [0,1] for now
        p0 = rand.random()
        p1 = 1-p0   
        r0 = s
        r1 = s
    else:'''
    sp = (s - C@s)/(1-np.vdot(s,s))
    #print(f'sp: {sp}')
    #print(f'sp^2: {np.vdot(sp,sp)}')
    #print(f'|sp - s|^2: {np.linalg.norm(sp-s)**2}')
    x = (1-np.vdot(s,s))/(1-np.vdot(sp,sp))
    #print(f'x: {x}')
    #if np.linalg.norm(sp-s)**2 < .03:
    if priors[0] == priors[1]:
        p0 = .5
        p1 = .5
        eigenvalues, eigenvectors = np.linalg.eig(C)
        #evals = np.array([abs(i) for i in eigenvalues])
        r_eigenvalues = np.array([i.real for i in eigenvalues if abs(i.imag)< .01])
        sort_indices = np.argsort(r_eigenvalues)
        sorted_eigenvalues = r_eigenvalues[sort_indices]
        sorted_eigenvectors = eigenvectors[:, sort_indices]
        eiv = sorted_eigenvalues[1:]
        eic = sorted_eigenvectors[:,1:]
        r0 = eic[:,0] + eic[:,1]  
        r1 = eic[:,1] - eic[:,0]
    else:
        if priors[0] < priors[1]:
            p0 = .5 - .5*np.sqrt(1-x)
            p1 = .5 + .5*np.sqrt(1-x)
            r0 = (2*p1*sp - s)/(p1-p0)
            r1 = (s - 2*p0*sp)/(p1-p0)
        elif priors[0] > priors[1]:
            p0 = .5 + .5*np.sqrt(1-x)
            p1 = .5 - .5*np.sqrt(1-x)
            r0 = (2*p1*sp - s)/(p1-p0)
            r1 = (s - 2*p0*sp)/(p1-p0)
            #r0 = (s - 2*p1*sp)/(p0-p1)
            #r1 = (2*p0*sp - s)/(p0-p1)
            
            
    #print(f'norm r0: {np.vdot(r0,r0)}, norm r1: {np.vdot(r1,r1)}')
    #print(f'r0: {r0}, r1: {r1}')
    r0 = r0/np.linalg.norm(r0)
    r1 = r1/np.linalg.norm(r1)
    #print(f'norm r0: {np.vdot(r0,r0)}, norm r1: {np.vdot(r1,r1)}')
    #print(f'r0r1: {np.vdot(r0,r1)}')
    #print(f'r0: {r0}, r1: {r1}')

    #extract pure states psi0,psi1 from rho_num and the bloch vectors r0,r1

    #all possible (positive) coeffs
    a1 = (r0[0]+1j*r0[1])*np.sqrt(.5*(1+r0[2]))/np.sqrt(1-r0[2]**2) #np.sqrt(.5*(1+r0[2]))
    a2 = np.sqrt(1-r0[2]**2)*np.sqrt(.5*(1-r0[2]))/(r0[0]+1j*r0[1]) #np.sqrt(.5*(1-r0[2]))
    #a3 = (r0[0]+1j*r0[1])*np.sqrt(.5*(1+r0[2]))/np.sqrt(1-r0[2]**2)
    #a4 = np.sqrt(1-r0[2]**2)*np.sqrt(.5*(1-r0[2]))/(r0[0]+1j*r0[1])
    a5 = (r1[0]+1j*r1[1])*np.sqrt(.5*(1+r1[2]))/np.sqrt(1-r1[2]**2) #np.sqrt(.5*(1+r1[2]))
    a6 = np.sqrt(1-r1[2]**2)*np.sqrt(.5*(1-r1[2]))/(r1[0]+1j*r1[1]) #np.sqrt(.5*(1-r1[2]))
    #a7 = (r1[0]+1j*r1[1])*np.sqrt(.5*(1+r1[2]))/np.sqrt(1-r1[2]**2)
    #a8 = np.sqrt(1-r1[2]**2)*np.sqrt(.5*(1-r1[2]))/(r1[0]+1j*r1[1])

   #lists of all possible coeffs, including megative and complex conjugates. each sublist correspons to a "physical sector" 
    v12 = [a1,a2,-a1,-a2]
    #v14 = [a1,a4,-a1,-a4]
    #v23 = [a2,a3,-a2,-a3]
    v56 = [a5,a6,-a5,-a6]
    #v58 = [a5,a8,-a5,-a8]
    #v67 = [a6,a7,-a6,-a7]

    pairs12 = [[i,j] for i in v12 for j in v12 if i!=j and i!= -j] 
    #pairs14 = [[i,j] for i in v14 for j in v14 if i!=j and i!= -j]
    #pairs23 = [[i,j] for i in v23 for j in v23 if i!=j and i!= -j]
    pairs56 = [[i,j] for i in v56 for j in v56 if i!=j and i!= -j]
    #pairs58 = [[i,j] for i in v58 for j in v58 if i!=j and i!= -j]
    #pairs67 = [[i,j] for i in v67 for j in v67 if i!=j and i!= -j]

    #pairs = pairs12 + pairs14 + pairs23 + pairs56 + pairs58 + pairs67
    pairs = pairs12 + pairs56

    #print(f'pairs: {pairs}')
    
    #all possible fidelities
    fid_psi0 = [abs(np.vdot(i, psi0))**2 for i in pairs]
    fid_psi1 = [abs(np.vdot(j, psi1))**2 for j in pairs]

    #Create the dictionaries for the fidelities and the corresponding num wavefunctions
    fid_psi0_dict = dict(zip(fid_psi0, pairs))
    fid_psi1_dict = dict(zip(fid_psi1, pairs))
    
    #find the max fidelity and the corresponding num wavefunction
    fid0 = max(fid_psi0_dict.keys())
    psi_num0 = fid_psi0_dict[fid0]
    fid1 = max(fid_psi1_dict.keys())
    psi_num1 = fid_psi1_dict[fid1]
            
    fid = [fid0,fid1]
    psin = [psi_num0,psi_num1]
    p = [p0,p1]

    #print(f'fid: {fid}, psin: {psin},  p: {p}')

    return([fid,psin,p])


In [ ]:
coeff = [0,1j,1j,0] 
priors = [.5,.5]
trials = [100*(i+1) for i in range(100)]

initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_x = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_c = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
output_psi = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[3] for i in tqdm(trials)]
output_p = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[4] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputx = list(map(list, zip(*output_x))) # transpose of output_x
outputc = list(map(list, zip(*output_c))) # transpose of output_c
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

fig_b, (ax1b, ax2b) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1b.plot(trials, outputx[0], label=r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
#ax1b.plot(trials , pfid, "--", label=r'$F=1$')
ax1b.set_xlabel(' N (trials)')
ax1b.set_ylabel(r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
ax1b.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax1b.legend(loc='best')
ax2b.plot(trials, outputx[1], label=r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
#ax2b.plot(trials , pfid, "--", label=r'$F=1$')
ax2b.set_xlabel(' N (trials)')
ax2b.set_ylabel(r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
ax2b.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
#ax2b.legend(loc='best')

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()

In [ ]:
coeff = [1,0,1,0] 
priors = [.5,.5]
trials = [100*(i+1) for i in range(100)]

initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_x = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_c = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
output_psi = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[3] for i in tqdm(trials)]
output_p = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[4] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputx = list(map(list, zip(*output_x))) # transpose of output_x
outputc = list(map(list, zip(*output_c))) # transpose of output_c
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

fig_b, (ax1b, ax2b) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1b.plot(trials, outputx[0], label=r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
#ax1b.plot(trials , pfid, "--", label=r'$F=1$')
ax1b.set_xlabel(' N (trials)')
ax1b.set_ylabel(r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
ax1b.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax1b.legend(loc='best')
ax2b.plot(trials, outputx[1], label=r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
#ax2b.plot(trials , pfid, "--", label=r'$F=1$')
ax2b.set_xlabel(' N (trials)')
ax2b.set_ylabel(r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
ax2b.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1{}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
#ax2b.legend(loc='best')

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()

In [ ]:
coeff = [1,0,.5**.5,.5**.5] 
priors = [.3,.7]
trials = [100*(i+1) for i in range(100)]
#trials = [10000]


initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_x = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_c = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
output_psi = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[3] for i in tqdm(trials)]
output_p = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[4] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputx = list(map(list, zip(*output_x))) # transpose of output_x
outputc = list(map(list, zip(*output_c))) # transpose of output_c
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

fig_b, (ax1b, ax2b) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1b.plot(trials, outputx[0], label=r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
#ax1b.plot(trials , pfid, "--", label=r'$F=1$')
ax1b.set_xlabel(' N (trials)')
ax1b.set_ylabel(r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
ax1b.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax1b.legend(loc='best')
ax2b.plot(trials, outputx[1], label=r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
#ax2b.plot(trials , pfid, "--", label=r'$F=1$')
ax2b.set_xlabel(' N (trials)')
ax2b.set_ylabel(r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
ax2b.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
#ax2b.legend(loc='best')

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()

In [ ]:
coeff = [1,0,1/(2**.5),1/(2**.5)] 
priors = [.7,.3]
trials = [100*(i+1) for i in range(100)]


initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_x = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_c = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
output_psi = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[3] for i in tqdm(trials)]
output_p = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[4] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputx = list(map(list, zip(*output_x))) # transpose of output_x
outputc = list(map(list, zip(*output_c))) # transpose of output_c
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

fig_b, (ax1b, ax2b) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1b.plot(trials, outputx[0], label=r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
#ax1b.plot(trials , pfid, "--", label=r'$F=1$')
ax1b.set_xlabel(' N (trials)')
ax1b.set_ylabel(r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
ax1b.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax1b.legend(loc='best')
ax2b.plot(trials, outputx[1], label=r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
#ax2b.plot(trials , pfid, "--", label=r'$F=1$')
ax2b.set_xlabel(' N (trials)')
ax2b.set_ylabel(r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
ax2b.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
#ax2b.legend(loc='best')

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()

In [ ]:
coeff = [(.5)**(.5),(.5)**(.5),(0.5)**(.5),(0.5)**(.5)] 
priors = [.3,.7]
trials = [100*(i+1) for i in range(100)]

initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_x = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_c = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
output_psi = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[3] for i in tqdm(trials)]
output_p = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[4] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputx = list(map(list, zip(*output_x))) # transpose of output_x
outputc = list(map(list, zip(*output_c))) # transpose of output_c
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

fig_b, (ax1b, ax2b) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1b.plot(trials, outputx[0], label=r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
#ax1b.plot(trials , pfid, "--", label=r'$F=1$')
ax1b.set_xlabel(' N (trials)')
ax1b.set_ylabel(r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
ax1b.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax1b.legend(loc='best')
ax2b.plot(trials, outputx[1], label=r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
#ax2b.plot(trials , pfid, "--", label=r'$F=1$')
ax2b.set_xlabel(' N (trials)')
ax2b.set_ylabel(r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
ax2b.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
#ax2b.legend(loc='best')

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()

In [ ]:
coeff = [(0)**(.5),(1)**(.5),1**(0.5)**(.5),0**(.5)]
priors = [.3,.7]
trials = [100*(i+1) for i in range(100)]

initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_x = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_c = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
output_psi = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[3] for i in tqdm(trials)]
output_p = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[4] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputx = list(map(list, zip(*output_x))) # transpose of output_x
outputc = list(map(list, zip(*output_c))) # transpose of output_c
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

fig_b, (ax1b, ax2b) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1b.plot(trials, outputx[0], label=r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
#ax1b.plot(trials , pfid, "--", label=r'$F=1$')
ax1b.set_xlabel(' N (trials)')
ax1b.set_ylabel(r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
ax1b.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax1b.legend(loc='best')
ax2b.plot(trials, outputx[1], label=r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
#ax2b.plot(trials , pfid, "--", label=r'$F=1$')
ax2b.set_xlabel(' N (trials)')
ax2b.set_ylabel(r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
ax2b.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
#ax2b.legend(loc='best')

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()

In [ ]:
#ENGLERT METHOD

coeff = [1,0,1/(2**.5),1/(2**.5)] 
priors = [.3,.7]
trials = [100*(i+1) for i in range(100)]

initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [englert_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_psi = [englert_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_p = [englert_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

ax1a.set_ylim(0,2)
ax2a.set_ylim(0,2)

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()

In [ ]:
coeff = [(.5)**(.5),1j*((.5)**(.5)),((.5)**(.5) + 1j*((.5)**(.5)))*(-(.5)**(.5)),0**(.5)]
priors = [.25,.75]
trials = [100*(i+1) for i in range(100)]

initial_states = Creating_states(Abstract=False, coeff = coeff)     # Creating the two states with these coefficients
psi0 = initial_states[0]
psi1 = initial_states[1]
scalar = abs(np.vdot(psi0,psi1))
print(f'overlap = {scalar}')

pfid  = [1 for i in trials]    # perfect fidelity of 1
output_f = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[0] for i in tqdm(trials)]
output_x = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[1] for i in tqdm(trials)]
output_c = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[2] for i in tqdm(trials)]
output_psi = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[3] for i in tqdm(trials)]
output_p = [experiment_fid(i,coeff,priors,POVM_elts, M_inv)[4] for i in tqdm(trials)]
outputf = list(map(list, zip(*output_f))) # transpose of output_f
outputx = list(map(list, zip(*output_x))) # transpose of output_x
outputc = list(map(list, zip(*output_c))) # transpose of output_c
outputpsi = list(map(list, zip(*output_psi))) # transpose of output_psi
outputp = list(map(list, zip(*output_p))) # transpose of output_p


fig_a, (ax1a, ax2a) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1a.plot(trials, outputf[0], label=r'$F_0=|\langle \psi_0|\psi_0^{\,n}\rangle|^2$')
ax1a.plot(trials , pfid, "--", label=r'$F=1$')
ax1a.set_xlabel(' N (trials)')
ax1a.set_ylabel(r'Fidelity $F$')
ax1a.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax1a.legend(loc='best')
ax2a.plot(trials, outputf[1], label=r'$F_1=|\langle \psi_1|\psi_1^{\,n}\rangle|^2$')
ax2a.plot(trials , pfid, "--", label=r'$F=1$')
ax2a.set_xlabel(' N (trials)')
ax2a.set_ylabel(r'Fidelity $F$')
ax2a.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
ax2a.legend(loc='best')

fig_b, (ax1b, ax2b) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1b.plot(trials, outputx[0], label=r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
#ax1b.plot(trials , pfid, "--", label=r'$F=1$')
ax1b.set_xlabel(' N (trials)')
ax1b.set_ylabel(r'$|\langle \xi|\psi_0\rangle\rangle|^2$')
ax1b.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
#ax1b.legend(loc='best')
ax2b.plot(trials, outputx[1], label=r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
#ax2b.plot(trials , pfid, "--", label=r'$F=1$')
ax2b.set_xlabel(' N (trials)')
ax2b.set_ylabel(r'$|\langle \xi|\psi_1\rangle\rangle|^2$')
ax2b.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))
#ax2b.legend(loc='best')

p0trials  = [priors[0] for i in trials] 
p1trials  = [priors[1] for i in trials] 
fig_c, (ax1c, ax2c) = plt.subplots(1,2, figsize=(10,4), sharey=True)
ax1c.plot(trials, outputp[0], label=r'$p_{0}$ numerical')
ax1c.plot(trials, p0trials, '--', label=r'$p_{0}$ theoretical')
ax2c.plot(trials, outputp[1], label=r'$p_{1}$ numerical')
ax2c.plot(trials, p1trials, '--', label=r'$p_{1}$ theoretical')
ax1c.set_xlabel(' N (trials)')
ax2c.set_xlabel(' N (trials)')
ax1c.set_ylabel(r'Probability')
ax2c.set_ylabel(r'Probability')
ax1c.legend(loc='best')
ax2c.legend(loc='best')
ax1c.set_title(r'$|\psi_0\rangle ={}|0\rangle+{}|1\rangle $, $p_0={}$'.format(roundc(coeff[0],1), roundc(coeff[1],1), priors[0]))
ax2c.set_title(r'$|\psi_1\rangle ={}|0\rangle+{}|1\rangle $, $p_1={}$'.format(roundc(coeff[2],1), roundc(coeff[3],1), priors[1]))

plt.show()